In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns',  8)

In [90]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [91]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,...,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,...,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,...,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,...,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,...,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,...,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [92]:

### Ваш код ###
X, y = data.drop(columns=['MEDV']), data['MEDV']

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)


2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [93]:

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
### Ваш код ###

linreg = LinearRegression()
linreg.fit(X_train,y_train)
linreg_pred = linreg.predict(X_test)


# Ridge
ridge_m= Ridge()
ridge_m.fit(X_train,y_train)
ridge_pred = ridge_m.predict(X_test)

# Lasso
lasso_m= Lasso()
lasso_m.fit(X_train,y_train)
lasso_pred = lasso_m.predict(X_test)

In [94]:
r2_lr = r2_score(y_test, linreg_pred)
r2_ridge = r2_score(y_test, ridge_pred)
r2_lasso = r2_score(y_test, lasso_pred)
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
results_regression

,model,task,R2
0,LR,task2,0.579218
1,Ridge,task2,0.571094
2,Lasso,task2,0.641457


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [95]:
### Ваш код ###
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV

params_alpha = [10**i for i in range(-5, 6)]

# RidgeCV
ridge_CV = RidgeCV(alphas = params_alpha)
ridge_CV.fit(X_train, y_train)
ridgeCV_pred = ridge_CV.predict(X_test)

# LassoCV
lasso_CV = LassoCV(alphas = params_alpha)
lasso_CV.fit(X_train, y_train)
lassoCV_pred = lasso_CV.predict(X_test)

#GridSearchCV
param_grid = {'alpha': params_alpha}
ridge_grid = Ridge()
lasso_grid = Lasso()
grid_ridge = GridSearchCV(estimator=ridge_grid, param_grid=param_grid, cv=5)
grid_ridge.fit(X_train, y_train)
grid_ridge_pred = grid_ridge.predict(X_test)
grid_lasso = GridSearchCV(estimator=lasso_grid, param_grid=param_grid, cv=5)
grid_lasso.fit(X_train, y_train)
grid_lasso_pred = grid_lasso.predict(X_test)



In [96]:
### Ваш код ###

r2_ridge_grid_search = r2_score(y_test, grid_ridge_pred)
r2_ridge_cv = r2_score(y_test, ridgeCV_pred)
r2_lasso_grid_search = r2_score(y_test, grid_lasso_pred)
r2_lasso_cv = r2_score(y_test, lassoCV_pred)
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
results_regression

,model,task,R2
0,LR,task2,0.579218
1,Ridge,task2,0.571094
2,Lasso,task2,0.641457
3,Ridge_GridSearchCV,task3,0.577935
4,RidgeCV,task3,0.577935
5,Lasso_GridSearchCV,task3,0.578951
6,LassoCV,task3,0.579215


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [98]:
# масштабирование выборки
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler,  MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Числовые признаки заполняем медианой и
simple_imputer = SimpleImputer(strategy='median')
std_scaler = StandardScaler()
pipe_num_std_scaler = Pipeline([('imputer', simple_imputer), ('scaler', std_scaler)])
col_transformer_std_scaler = ColumnTransformer([('num_preproc', pipe_num_std_scaler, [x for x in X_train.columns if X_train[x].dtype!='object'])])

X_train_std_scaler = col_transformer_std_scaler.fit_transform(X_train)
X_test_std_scaler= col_transformer_std_scaler.transform(X_test)

minmax_scaler = MinMaxScaler()
pipe_num_minmax_scaler = Pipeline([('imputer', simple_imputer), ('scaler', minmax_scaler)])
col_transformer_minmax_scaler = ColumnTransformer([('num_preproc', pipe_num_minmax_scaler, [x for x in X_train.columns if X_train[x].dtype!='object'])])

X_train_minmax_scaler = col_transformer_minmax_scaler.fit_transform(X_train)
X_test_minmax_scaler = col_transformer_minmax_scaler.transform(X_test)

In [99]:
# Ridge
ridge_std_scaler = Ridge()
ridge_std_scaler.fit(X_train_std_scaler,y_train)
ridge_std_scaler_pred = ridge_std_scaler.predict(X_test_std_scaler)

ridge_minmax_scaler = Ridge()
ridge_minmax_scaler.fit(X_train_minmax_scaler,y_train)
ridge_minmax_scaler_pred = ridge_minmax_scaler.predict(X_test_minmax_scaler)

# Lasso
lasso_std_scaler = Lasso()
lasso_std_scaler.fit(X_train_std_scaler,y_train)
lasso_std_scaler_pred = lasso_std_scaler.predict(X_test_std_scaler)

lasso_minmax_scaler = Lasso()
lasso_minmax_scaler.fit(X_train_minmax_scaler,y_train)
lasso_minmax_scaler_pred = lasso_minmax_scaler.predict(X_test_minmax_scaler)

### Ваш код ###

r2_ridge_standart_scaler = r2_score(y_test, ridge_std_scaler_pred)
r2_ridge_min_max_scaler = r2_score(y_test, ridge_minmax_scaler_pred)
r2_lasso_standart_scaler = r2_score(y_test, lasso_std_scaler_pred)
r2_lasso_min_max_scaler = r2_score(y_test, lasso_minmax_scaler_pred)
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [100]:
# RidgeCV
ridge_CV_std_scaler= RidgeCV(alphas = params_alpha)
ridge_CV_std_scaler.fit(X_train_std_scaler, y_train)
ridge_CV_std_scaler_pred = ridge_CV_std_scaler.predict(X_test_std_scaler)

ridge_CV_minmax_scaler= RidgeCV(alphas = params_alpha)
ridge_CV_minmax_scaler.fit(X_train_minmax_scaler, y_train)
ridge_CV_minmax_scaler_pred = ridge_CV_minmax_scaler.predict(X_test_minmax_scaler)

# LassoCV
lasso_CV_std_scaler= LassoCV(alphas = params_alpha)
lasso_CV_std_scaler.fit(X_train_std_scaler, y_train)
lasso_CV_std_scaler_pred = lasso_CV_std_scaler.predict(X_test_std_scaler)

lasso_CV_minmax_scaler= LassoCV(alphas = params_alpha)
lasso_CV_minmax_scaler.fit(X_train_minmax_scaler, y_train)
lasso_CV_minmax_scaler_pred = lasso_CV_minmax_scaler.predict(X_test_minmax_scaler)


In [101]:


### Ваш код ###

r2_ridge_standart_scaler_cv = r2_score(y_test, ridge_CV_std_scaler_pred)
r2_ridge_min_max_scaler_cv = r2_score(y_test, ridge_CV_minmax_scaler_pred)
r2_lasso_standart_scaler_cv = r2_score(y_test, lasso_CV_std_scaler_pred)
r2_lasso_min_max_scaler_cv = r2_score(y_test, lasso_CV_minmax_scaler_pred)
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [102]:
from sklearn.preprocessing import PolynomialFeatures
#PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)

X_train_std_scaler_poly  = polynomial_converter.fit_transform(X_train_std_scaler)
X_test_std_scaler_poly  = polynomial_converter.transform(X_test_std_scaler)

X_train_minmax_scaler_poly = polynomial_converter.fit_transform(X_train_minmax_scaler)
X_test_minmax_scaler_poly = polynomial_converter.transform(X_test_minmax_scaler)

In [103]:
# Ridge
ridge_std_scaler_poly = Ridge()
ridge_std_scaler_poly.fit(X_train_std_scaler_poly,y_train)
ridge_std_scaler_poly_pred = ridge_std_scaler_poly.predict(X_test_std_scaler_poly)

ridge_minmax_scaler_poly = Ridge()
ridge_minmax_scaler_poly.fit(X_train_minmax_scaler_poly,y_train)
ridge_minmax_scaler_poly_pred = ridge_minmax_scaler_poly.predict(X_test_minmax_scaler_poly)

# Lasso
lasso_std_scaler_poly = Lasso()
lasso_std_scaler_poly.fit(X_train_std_scaler_poly,y_train)
lasso_std_scaler_poly_pred = lasso_std_scaler_poly.predict(X_test_std_scaler_poly)

lasso_minmax_scaler_poly = Lasso()
lasso_minmax_scaler_poly.fit(X_train_minmax_scaler_poly,y_train)
lasso_minmax_scaler_poly_pred = lasso_minmax_scaler_poly.predict(X_test_minmax_scaler_poly)

### Ваш код ###

r2_ridge_standart_scaler_poly = r2_score(y_test, ridge_std_scaler_poly_pred)
r2_ridge_min_max_scaler_poly = r2_score(y_test, ridge_minmax_scaler_poly_pred)
r2_lasso_standart_scaler_poly = r2_score(y_test, lasso_std_scaler_poly_pred)
r2_lasso_min_max_scaler_poly = r2_score(y_test, lasso_minmax_scaler_poly_pred)
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
results_regression

,model,task,R2
0,LR,task2,0.579218
1,Ridge,task2,0.571094
2,Lasso,task2,0.641457
3,Ridge_GridSearchCV,task3,0.577935
4,RidgeCV,task3,0.577935
5,Lasso_GridSearchCV,task3,0.578951
6,LassoCV,task3,0.579215
7,Ridge_StandardScaler,task4,0.578717
8,Ridge_MinMaxScaler,task4,0.599453
9,Lasso_StandardScaler,task4,0.510293


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [104]:

# RidgeCV
ridge_CV_std_scaler_poly= RidgeCV(alphas = params_alpha)
ridge_CV_std_scaler_poly.fit(X_train_std_scaler_poly, y_train)
ridge_CV_std_scaler_poly_pred = ridge_CV_std_scaler_poly.predict(X_test_std_scaler_poly)

ridge_CV_minmax_scaler_poly= RidgeCV(alphas = params_alpha)
ridge_CV_minmax_scaler_poly.fit(X_train_minmax_scaler_poly, y_train)
ridge_CV_minmax_scaler_poly_pred = ridge_CV_minmax_scaler_poly.predict(X_test_minmax_scaler_poly)

# LassoCV
lasso_CV_std_scaler_poly= LassoCV(alphas = params_alpha)
lasso_CV_std_scaler_poly.fit(X_train_std_scaler_poly, y_train)
lasso_CV_std_scaler_poly_pred = lasso_CV_std_scaler_poly.predict(X_test_std_scaler_poly)

lasso_CV_minmax_scaler_poly= LassoCV(alphas = params_alpha)
lasso_CV_minmax_scaler_poly.fit(X_train_minmax_scaler_poly, y_train)
lasso_CV_minmax_scaler_poly_pred = lasso_CV_minmax_scaler_poly.predict(X_test_minmax_scaler_poly)

### Ваш код ###

r2_ridge_standart_scaler_poly_cv = r2_score(y_test, ridge_CV_std_scaler_poly_pred)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, ridge_CV_minmax_scaler_poly_pred)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, lasso_CV_std_scaler_poly_pred)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, lasso_CV_minmax_scaler_poly_pred)
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
results_regression

,model,task,R2
0,LR,task2,0.579218
1,Ridge,task2,0.571094
2,Lasso,task2,0.641457
3,Ridge_GridSearchCV,task3,0.577935
4,RidgeCV,task3,0.577935
5,Lasso_GridSearchCV,task3,0.578951
6,LassoCV,task3,0.579215
7,Ridge_StandardScaler,task4,0.578717
8,Ridge_MinMaxScaler,task4,0.599453
9,Lasso_StandardScaler,task4,0.510293


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [105]:
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", category=ConvergenceWarning)


pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', StandardScaler()),
                     ('polynomial', PolynomialFeatures()),
                     ('regression', Lasso())])

### Ваш код ###

parameters = {'scaler':[StandardScaler(), MinMaxScaler()],
              'polynomial__degree':np.arange(1,3,1),
              'regression':[Lasso(), Ridge()],
              'regression__alpha':np.logspace(-2, 2, 5)}
select_model = GridSearchCV(pipeline, parameters, scoring='r2', n_jobs=-1, verbose=10 )
select_model.fit(X_train, y_train)
select_model.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5; 1/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 2/5; 1/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 3/5; 1/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 4/5; 1/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 2/5; 1/40] END polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler();, score=0.678 total time=   0.0s
[CV 3/5; 1/40] END polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler();, score=0.716 total time=   0.0s
[CV 5/5; 1/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 1/5; 2/40] START polynomial__degree=1, regression=Lasso(), regression__alpha=0.01, scaler=MinMa

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.401e+01, tolerance: 2.763e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.766e+00, tolerance: 2.824e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.405e+01, tolerance: 2.765e

[CV 5/5; 21/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler();, score=0.893 total time=   0.1s
[CV 1/5; 22/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=0.01, scaler=MinMaxScaler()
[CV 2/5; 19/40] END polynomial__degree=1, regression=Ridge(), regression__alpha=100.0, scaler=StandardScaler();, score=0.636 total time=   0.0s
[CV 3/5; 19/40] START polynomial__degree=1, regression=Ridge(), regression__alpha=100.0, scaler=StandardScaler()
[CV 4/5; 23/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=0.1, scaler=StandardScaler();, score=0.878 total time=   0.0s
[CV 5/5; 20/40] END polynomial__degree=1, regression=Ridge(), regression__alpha=100.0, scaler=MinMaxScaler();, score=0.392 total time=   0.0s
[CV 5/5; 23/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=0.1, scaler=StandardScaler()
[CV 1/5; 21/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=0.01, scaler=

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.199e+00, tolerance: 2.765e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.244e+00, tolerance: 2.763e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.308e+01, tolerance: 2.782e

[CV 3/5; 21/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler();, score=0.880 total time=   0.1s
[CV 4/5; 21/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=0.01, scaler=StandardScaler()
[CV 5/5; 25/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=1.0, scaler=StandardScaler();, score=0.848 total time=   0.0s
[CV 5/5; 26/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=1.0, scaler=MinMaxScaler();, score=0.287 total time=   0.0s
[CV 1/5; 26/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=1.0, scaler=MinMaxScaler()
[CV 1/5; 27/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=10.0, scaler=StandardScaler()
[CV 1/5; 26/40] END polynomial__degree=2, regression=Lasso(), regression__alpha=1.0, scaler=MinMaxScaler();, score=0.354 total time=   0.0s
[CV 2/5; 26/40] START polynomial__degree=2, regression=Lasso(), regression__alpha=1.0, scaler=MinMaxSc

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler()),
                ('polynomial', PolynomialFeatures()),
                ('regression', Ridge(alpha=0.1))])

In [106]:
select_model_pred = select_model.predict(X_test)
best_params = select_model.best_params_
print('Параметры лучшей модели:\n', best_params)
r2_best_model = r2_score(y_test, select_model_pred)
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Параметры лучшей модели:
 {'polynomial__degree': 2, 'regression': Ridge(), 'regression__alpha': 0.1, 'scaler': MinMaxScaler()}


In [107]:
results_regression

,model,task,R2
0,LR,task2,0.579218
1,Ridge,task2,0.571094
2,Lasso,task2,0.641457
3,Ridge_GridSearchCV,task3,0.577935
4,RidgeCV,task3,0.577935
5,Lasso_GridSearchCV,task3,0.578951
6,LassoCV,task3,0.579215
7,Ridge_StandardScaler,task4,0.578717
8,Ridge_MinMaxScaler,task4,0.599453
9,Lasso_StandardScaler,task4,0.510293


http://archive.ics.uci.edu/ml/datasets/Adult

In [108]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,...,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,...,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,...,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,...,0,40,United-States,<=50K
3,53,Private,234721,11th,...,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,...,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [109]:
data = pd.read_csv('adult.csv')
data.head()

### Ваш код ###
X, y = data.drop(columns=['class']), (data['class'] == "<=50K").astype(int)

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)


10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [110]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

Dummy = DummyClassifier(strategy="most_frequent")
Dummy.fit(X_train, y_train)

### Ваш код ###

f1_most_frequent = f1_score(y_test, Dummy.predict(X_test))
acc_most_frequent = accuracy_score(y_test, Dummy.predict(X_test))
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.860891,0.755758


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [111]:
### Ваш код ###

simple_imputer = SimpleImputer(strategy='median')
pipe_imputer = Pipeline([('imputer', simple_imputer)])
col_transformer_null = ColumnTransformer([('num_preproc', pipe_num_std_scaler, [x for x in X_train.columns if X_train[x].dtype!='object'])])

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [112]:
categirical_col = [x for x in X_train.columns if X_train[x].dtype=='object']
numeric_col = [x for x in X_train.columns if X_train[x].dtype!='object']

### Ваш код ###

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [113]:
simple_imputer = SimpleImputer(strategy='median')
std_scaler =MinMaxScaler()
pipe_num_minmax_scaler = Pipeline([('imputer', simple_imputer), ('scaler', MinMaxScaler())])


s_imputer_cat = SimpleImputer(strategy='constant', fill_value='unknown')
ohe_encoder_cat = OneHotEncoder(handle_unknown='ignore')
pipe_cat_one_hot= Pipeline([('imputer', s_imputer_cat), ('encoder', ohe_encoder_cat)])

transformer_cat_num= ColumnTransformer([('num_preproc', pipe_num_minmax_scaler, numeric_col),
                                         ('cat_preproc', pipe_cat_one_hot, categirical_col)])


X_train_col_transform  = transformer_cat_num.fit_transform(X_train)
X_test_col_transform   = transformer_cat_num.transform(X_test)


In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

#LogisticRegression
Log_Reg = LogisticRegression(max_iter=1000)
Log_Reg.fit(X_train_col_transform, y_train)
Log_Reg_pred = Log_Reg.predict(X_test_col_transform)


# KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train_col_transform, y_train)
KNN_pred = KNN.predict(X_test_col_transform)

#LinearSVC
L_SVC = LinearSVC()
L_SVC.fit(X_train_col_transform, y_train)
L_SVC_pred = L_SVC.predict(X_test_col_transform)
### Ваш код ###

f1_LR =  cross_val_score(Log_Reg, X_test_col_transform, y_test, scoring='f1').mean()
acc_LR = cross_val_score(Log_Reg, X_test_col_transform, y_test, scoring='accuracy').mean()
f1_KNN = cross_val_score(KNN, X_test_col_transform, y_test, scoring='f1').mean()
acc_KNN = cross_val_score(KNN, X_test_col_transform, y_test, scoring='accuracy').mean()
f1_SVM = cross_val_score(L_SVC, X_test_col_transform, y_test, scoring='f1').mean()
acc_SVM = cross_val_score(L_SVC, X_test_col_transform, y_test, scoring='accuracy').mean()
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.860891,0.755758
1,LogisticRegression,task13,0.897015,0.839493
2,KNeighborsClassifier,task13,0.831256,0.763743
3,LinearSVC,task13,0.900520,0.845328


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [115]:
s_imputer_cat = SimpleImputer(missing_values="?", strategy='most_frequent', fill_value='unknown')
s_imputer_cat.fit([["?", "ert","RDFGT"], ["ert", "dfert", "defrt"], ["ert", "fd", "?"]])
X = [["?", "ffg", "?"], ["dg5", "fddfg", "ftrf"], ["?", "fhuye", "gdie"]]
print(s_imputer_cat.transform(X))

[['ert' 'ffg' 'RDFGT']
 ['dg5' 'fddfg' 'ftrf']
 ['ert' 'fhuye' 'gdie']]


In [116]:
simple_imputer = SimpleImputer(strategy='median')
std_scaler =MinMaxScaler()
pipe_num_minmax_scaler = Pipeline([('imputer', simple_imputer), ('scaler', MinMaxScaler())])


s_imputer_cat_freq = SimpleImputer(missing_values="?", strategy='most_frequent', fill_value='unknown')
ohe_encoder_cat = OneHotEncoder(handle_unknown='ignore')
pipe_cat_one_hot_freq= Pipeline([('imputer', s_imputer_cat_freq), ('encoder', ohe_encoder_cat)])

transformer_cet_num_freq = ColumnTransformer([('num_preproc', pipe_num_minmax_scaler, numeric_col),
                                         ('cat_preproc', pipe_cat_one_hot_freq, categirical_col)])


X_train_col_transform_freq  = transformer_cet_num_freq.fit_transform(X_train)
X_test_col_transform_freq   = transformer_cet_num_freq.transform(X_test)

In [117]:
#LogisticRegression
Log_Reg = LogisticRegression(max_iter=1000)
Log_Reg.fit(X_train_col_transform_freq , y_train)
Log_Reg_pred = Log_Reg.predict(X_test_col_transform_freq)


# KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train_col_transform_freq, y_train)
KNN_pred = KNN.predict(X_train_col_transform_freq)

#LinearSVC
L_SVC = LinearSVC()
L_SVC.fit(X_train_col_transform_freq, y_train)
L_SVC_pred = L_SVC.predict(X_train_col_transform_freq)
### Ваш код ###


### Ваш код ###

In [118]:
f1_LR_freq =  cross_val_score(Log_Reg, X_test_col_transform_freq, y_test, scoring='f1').mean()
acc_LR_freq = cross_val_score(Log_Reg, X_test_col_transform_freq, y_test, scoring='accuracy').mean()
f1_KNN_freq  = cross_val_score(KNN, X_test_col_transform_freq, y_test, scoring='f1').mean()
acc_KNN_freq = cross_val_score(KNN, X_test_col_transform_freq, y_test, scoring='accuracy').mean()
f1_SVM_freq = cross_val_score(L_SVC, X_test_col_transform_freq, y_test, scoring='f1').mean()
acc_SVM_freq = cross_val_score(L_SVC, X_test_col_transform_freq, y_test, scoring='accuracy').mean()
results_classification.loc[4] = ['LogisticRegression_impute_freq', 'task14', f1_LR_freq, acc_LR_freq]
results_classification.loc[5] = ['KNeighborsClassifier_impute_freq', 'task14', f1_KNN_freq, acc_KNN_freq]
results_classification.loc[6] = ['LinearSVC_impute_freq', 'task14', f1_SVM_freq, acc_SVM_freq]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [119]:
data_del = data.copy()
data_del = data_del.replace({'?': np.nan}).dropna()

X_del, y_del = data_del.drop(columns=['class']), (data_del['class'] == "<=50K").astype(int)

X_train_del, X_test_del, y_train_del, y_test_del = train_test_split(X_del, y_del, test_size=0.2, random_state=7)


In [120]:

X_train_col_transform_del  = transformer_cet_num_freq.fit_transform(X_train_del)
X_test_col_transform_del   = transformer_cet_num_freq.transform(X_test_del)


#LogisticRegression
Log_Reg_del = LogisticRegression(max_iter=1000)
Log_Reg_del.fit(X_train_col_transform_del , y_train_del)
Log_Reg_del_pred = Log_Reg_del.predict(X_test_col_transform_del)


# KNeighborsClassifier
KNN_del = KNeighborsClassifier(n_neighbors=2)
KNN_del.fit(X_train_col_transform_del, y_train_del)
KNN_pred = KNN_del.predict(X_test_col_transform_del)

#LinearSVC
L_SVC_del = LinearSVC()
L_SVC_del.fit(X_train_col_transform_del, y_train_del)
L_SVC_del_pred = L_SVC_del.predict(X_test_col_transform_del)
### Ваш код ###


### Ваш код ###

f1_LR_del_missings = cross_val_score(Log_Reg_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
acc_LR_del_missings = cross_val_score(Log_Reg_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
f1_KNN_del_missings = cross_val_score(KNN_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
acc_KNN_del_missings = cross_val_score(KNN_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
f1_SVM_del_missings = cross_val_score(L_SVC_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
acc_SVM_del_missings = cross_val_score(L_SVC_del, X_test_col_transform_del, y_test_del, scoring='f1').mean()
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

In [121]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.860891,0.755758
1,LogisticRegression,task13,0.897015,0.839493
2,KNeighborsClassifier,task13,0.831256,0.763743
3,LinearSVC,task13,0.900520,0.845328
4,LogisticRegression_impute_freq,task14,0.898512,0.841744
5,KNeighborsClassifier_impute_freq,task14,0.830575,0.763231
6,LinearSVC_impute_freq,task14,0.899418,0.843383
7,LogisticRegression_delete_missings,task15,0.897963,0.897963
8,KNeighborsClassifier_delete_missings,task15,0.830033,0.830033
9,LinearSVC_delete_missings,task15,0.900462,0.900462


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [123]:

# RandomForestClassifier

RF = RandomForestClassifier(max_depth=2, random_state=7).fit(X_train_col_transform_freq, y_train)
RF_pred = RF.predict(X_test_col_transform_freq)


# GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=7).fit(X_train_col_transform_freq, y_train)
GB_pred = GB.predict(X_test_col_transform_freq)

### Ваш код ###

f1_RF = cross_val_score(RF, X_test_col_transform_freq, y_test, cv=3, scoring='f1').mean()
acc_RF = cross_val_score(RF, X_test_col_transform_freq, y_test, cv=3, scoring='accuracy').mean()
f1_GB = cross_val_score(GB, X_test_col_transform_freq, y_test, cv=3, scoring='accuracy').mean()
acc_GB = cross_val_score(GB, X_test_col_transform_freq, y_test, cv=3, scoring='f1').mean()
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]
results_classification


,model,task,f1,accuracy
0,Most Frequent class,task10,0.860891,0.755758
1,LogisticRegression,task13,0.897015,0.839493
2,KNeighborsClassifier,task13,0.831256,0.763743
3,LinearSVC,task13,0.900520,0.845328
4,LogisticRegression_impute_freq,task14,0.898512,0.841744
5,KNeighborsClassifier_impute_freq,task14,0.830575,0.763231
6,LinearSVC_impute_freq,task14,0.899418,0.843383
7,LogisticRegression_delete_missings,task15,0.897963,0.897963
8,KNeighborsClassifier_delete_missings,task15,0.830033,0.830033
9,LinearSVC_delete_missings,task15,0.900462,0.900462


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [124]:
data = pd.read_csv('adult.csv')
data.head()

### Ваш код ###
X, y = data.drop(columns=['class']), (data['class'] == "<=50K").astype(int)
X = X.replace('?', np.nan)
# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [125]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer

# transformer_cat_num   : num - mean and std,   cat - fillna + onehot
# transformer_cet_num_freq : num - mean and std,   cat - ? -> freq + onehot

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler()],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__cat__onehot__handle_unknown': ['ignore', 'error']
}


In [126]:
best_params = {}
f1_best = 0
acc_best = 0

for model_name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=2,
        scoring={'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)},
        refit='f1',
        n_jobs=-1
    )

    grid_search.fit(X_train, y_train)

    best_params[model_name] = grid_search.best_params_
    f1_score_mean = grid_search.best_score_
    acc_score_mean = cross_val_score(grid_search.best_estimator_, X_test, y_test, cv=2, scoring='accuracy').mean()

    current_results = pd.DataFrame({
        'model': [model_name],
        'task': ['task17'],
        'f1': [f1_score_mean],
        'accuracy': [acc_score_mean]
    })


    results_classification = pd.concat([results_classification, current_results], ignore_index=True)

print('Параметры лучшей модели:\n', best_params)
print(results_classification)

/usr/local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/site-packages/sklearn/utils/_response.py", line 211, in _get_response_values
    y_pred = prediction_method(X)
  File "/usr/local/lib/python3.10/site-packages/sklearn/pipeline.py", line 600, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 316, in wrapped
 

Параметры лучшей модели:
 {'RandomForestClassifier': {'preprocessor__cat__imputer__strategy': 'constant', 'preprocessor__cat__onehot__handle_unknown': 'ignore', 'preprocessor__num__imputer__strategy': 'median', 'preprocessor__num__scaler': StandardScaler()}, 'GradientBoostingClassifier': {'preprocessor__cat__imputer__strategy': 'constant', 'preprocessor__cat__onehot__handle_unknown': 'ignore', 'preprocessor__num__imputer__strategy': 'most_frequent', 'preprocessor__num__scaler': MinMaxScaler()}}
                                   model    task        f1  accuracy
0                    Most Frequent class  task10  0.860891  0.755758
1                     LogisticRegression  task13  0.897015  0.839493
2                   KNeighborsClassifier  task13  0.831256  0.763743
3                              LinearSVC  task13  0.900520  0.845328
4         LogisticRegression_impute_freq  task14  0.898512  0.841744
5       KNeighborsClassifier_impute_freq  task14  0.830575  0.763231
6                

In [ ]:
results_classification

In [ ]:
results_classification